In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##Some Setup

In [2]:
from skimage import io , transform, color
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



cpu


In [3]:
# part2_path = './patched_pa2_data/pa2_data 2/part2_data/'
part2_path = '/content/drive/My Drive/Colab/patched_pa2_data/pa2_data 2/part2_data/'
print(part2_path + "train.csv")
train_df = pd.read_csv(part2_path + "train.csv")
val_df = pd.read_csv(part2_path + "validation.csv")
test_df = pd.read_csv(part2_path + "test.csv")
sub_df = pd.read_csv(part2_path + "submission.csv")
print(train_df.iloc[0])
print(val_df.iloc[0])
print(test_df.iloc[0])
print(sub_df.iloc[0])

/content/drive/My Drive/Colab/patched_pa2_data/pa2_data 2/part2_data/train.csv
label                                                    1
text     I guess those who have been in a one-sided rel...
Name: 0, dtype: object
label                                                    1
text     This is one of the movies of Dev Anand who gav...
Name: 0, dtype: object
label                                                    1
text     This movie is intelligent. That is, more than ...
Name: 0, dtype: object
label                                                    3
text     Less self-conscious and much less pretentious ...
Name: 0, dtype: object


In [4]:
train_df

,label,text
0,1,I guess those who have been in a one-sided rel...
1,1,Almost 30 years later I recall this original P...
2,1,This movie is one of the only historical docum...
3,0,This movie has made me upset! When I think of ...
4,0,I remember this movie when i was 13 seems a lo...
...,...,...
39995,0,Calling this a Sunday School movie might be ge...
39996,1,Convicts is very much a third act sort of film...
39997,1,In the Mood for Love a teasing allegory of lon...
39998,0,The fallen ones falls under the waste of life ...


In [0]:
# class TabularDataset(Dataset):
  
#     def __init__(self, dataFrame, trainer = False, ):
#         # self.file_path = file_path
#         # self.classes = [0, 1]
#         self.df = dataFrame

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         X = self.df.iloc[idx,1]
#         y = self.df.iloc[idx, 0]
#         return X , y

#     def __len__(self):
#         return len(self.df)

# train_set = TabularDataset(train_df)
# val_set = TabularDataset(val_df)
# test_set = TabularDataset(test_df)
# sub_set = TabularDataset(sub_df)


In [5]:
from torchtext import *
from torchtext.data import *

import nltk
nltk.download('punkt')
from nltk import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:

txt_field = data.Field(tokenize=word_tokenize, lower=True, include_lengths=True, batch_first=True)
label_field = data.Field(sequential=False, use_vocab=False, batch_first=True)

# make splits for data
train, val, test = TabularDataset.splits(path=part2_path , train='train.csv', validation="validation.csv", test='test.csv',format='csv', 
                                  fields=[('label', label_field),('text', txt_field)], skip_header=True)

# build the vocabulary on the training set only
txt_field.build_vocab(train, min_freq=5)
# label_field.build_vocab(train)



In [0]:
# make iterator for splits
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_sizes=(32, 256, 256),
    sort_key=lambda x: len(x.text),sort_within_batch=True, device=device)
# train_iter, test_iter = data.BucketIterator.splits((train, val), batch_size=30, 
                                                  #  sort_key=lambda x: len(x.sentence),sort_within_batch=True)

In [10]:
print(f'Number of training samples: {len(train.examples)}')
print(f'Number of validation samples: {len(val.examples)}')
print(f'Number of testing samples: {len(test.examples)}')

print(f'Example of training data:\n {vars(train.examples[0])}\n')
print(f'Example of validation data:\n {vars(val.examples[1])}\n')
print(f'Example of testing data:\n {vars(test.examples[1])}\n')



Number of training samples: 40000
Number of validation samples: 5000
Number of testing samples: 3000
Example of training data:
 {'label': '1', 'text': ['i', 'guess', 'those', 'who', 'have', 'been', 'in', 'a', 'one-sided', 'relationship', 'of', 'some', 'sort', 'before', 'will', 'be', 'able', 'identify', 'with', 'the', 'lead', 'character', 'minako', 'yuko', 'tanaka', ',', 'a', '50', 'year', 'old', 'woman', 'who', 'is', 'still', 'in', 'the', 'pink', 'of', 'good', 'health', ',', 'as', 'demonstrated', 'by', 'her', 'daily', ',', 'grinding', 'routine', 'of', 'waking', 'up', 'extremely', 'early', 'in', 'the', 'morning', 'to', 'prepare', 'for', 'her', 'milk', 'delivery', 'work', ',', 'where', 'she', 'has', 'to', 'lug', 'bottles', 'of', 'megmilk', 'in', 'a', 'bag', 'in', 'a', 'route', 'around', 'her', 'town', 'like', 'clockwork', ',', 'to', 'exchange', 'empty', 'bottles', 'for', 'full', 'ones', ',', 'and', 'to', 'collect', 'payment', 'and', 'issue', 'receipt', '.', 'and', 'there', "'s", 'always'

In [11]:
_, batch = next(enumerate(train_iter))
print('label tensor', batch.label.shape)
print(batch.label)
print()
sent, sent_len = batch.text
print('sentence length tensor', sent_len.shape)
print(sent_len)
print()
print('sentence tensor', sent.shape)
print(sent)

label tensor torch.Size([32])
tensor([1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1])

sentence length tensor torch.Size([32])
tensor([219, 219, 219, 219, 218, 218, 218, 218, 218, 218, 218, 217, 217, 217,
        217, 217, 217, 217, 217, 217, 217, 217, 216, 216, 216, 216, 216, 216,
        216, 216, 216, 216])

sentence tensor torch.Size([32, 219])
tensor([[  53,   38, 1155,  ...,  907,   22,    4],
        [  53,   15,  251,  ...,    6,  170,   33],
        [  12,  221,   13,  ...,   45, 2243,   33],
        ...,
        [  56,   23,  111,  ...,    1,    1,    1],
        [  42,    2, 3827,  ...,    1,    1,    1],
        [   2, 1423,    7,  ...,    1,    1,    1]])


In [0]:
def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer, save_path):
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss


def TRAIN(model, train_loader, valid_loader,  num_epochs, eval_every, total_step, criterion, optimizer, val_loss, device, save_name):
    loss_his = {'train' : [] , 'val' : []}
    acc_his = {'train' : [] , 'val' : []}
    
    running_loss = 0.0
    running_cor = 0
    running_num = 0
    global_step = 0
    if val_loss==None:
        best_val_loss = float("Inf")  
    else: 
        best_val_loss=val_loss
    
    model.to(device)
    for epoch in range(num_epochs):  # loop over the dataset multiple times

        for i, X in enumerate(train_loader):
            model.train()

            inputs = (X.text[0]).to(device)
            sent_len = (X.text[1]).to(device)
            labels = X.label.to(device)

            '''Training of the model'''
            # Forward pass
            outputs = model(inputs,sent_len)
            if(not(  (outputs.reshape(-1) >= 0).all() and (outputs.reshape(-1) <= 1).all() ) ):
              print("Something Wrong in the model output!")
              return X, outputs
            loss = criterion(outputs, labels.reshape(-1,1).float())
            predicts = torch.where(outputs.reshape(-1) > 0.5, torch.ones(1).to(device) ,torch.zeros(1).to(device))
            running_cor += torch.sum(predicts == labels)
            running_num += len(labels)
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            global_step += 1

            running_loss += loss.item()

            '''Evaluating the model every x steps'''
            if global_step % eval_every == 0:
                with torch.no_grad():
                    model.eval()
                    val_running_loss = 0.0
                    val_running_cor = 0
                    val_running_num = 0
                    for val_X in valid_loader:
                        val_labels = val_X.label
                        val_outputs = model(val_X.text[0], val_X.text[1])
                        val_loss = criterion(val_outputs, val_labels.reshape(-1,1).float())
                        val_running_loss += val_loss.item()
                        predicts = torch.where(val_outputs.reshape(-1) > 0.5,torch.ones(1).to(device),torch.zeros(1).to(device))
                        val_running_cor += torch.sum(predicts == val_labels)
                        val_running_num += len(val_labels)
                    average_train_loss = running_loss / eval_every
                    average_val_loss = val_running_loss / len(valid_loader)

                    average_train_acc = running_cor / float(running_num)
                    # print(val_running_cor,float(len(valid_loader)))
                    average_val_acc = val_running_cor / float(val_running_num)

                    print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}' 
                          .format(epoch+1, num_epochs, global_step, total_step, average_train_loss, average_val_loss))
                    print(f'Train Acc : {average_train_acc} ; Validation Acc {average_val_acc}')

                    loss_his['train'].append(average_train_loss)
                    loss_his['val'].append(average_val_loss)
                    acc_his['train'].append(average_train_acc)
                    acc_his['val'].append(average_val_acc)


                    running_loss = 0.0
                    running_num = 0
                    running_cor = 0
                    if average_val_loss < best_val_loss:
                        best_val_loss = average_val_loss
                        save_checkpoint(save_name, model, optimizer, best_val_loss)
                    

            
    print('Finished Training')
    return loss_his, acc_his

##RNN Baseline Model

In [0]:
import torch.nn as nn

class Text_RNN(nn.Module):
    def __init__(self, n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features):
        super(Text_RNN, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_vocab, embedding_dim)
        self.rnn = nn.RNN(
                input_size=embedding_dim,
                hidden_size=n_hidden,
                num_layers=n_layers,
                batch_first=True,
                dropout = dropout,
                nonlinearity = 'relu'
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_hidden, out_features)
        self.sigmoid = nn.Sigmoid()
    def forward(self, sent, sent_len):
        # sent: batch_size, max_sent_len
        # sent_len: batch_size
        # print(sent.dtype)
        sent_emb = self.emb(sent)  #batch_size, max_sent_len, embedding_dim
        # method 1
        # rnn_outputs, hidden = self.rnn(sent_emb)
        # print("here", outputs)
        # method 2, pack the input sequence, more computationally efficient
        packed_embedded = nn.utils.rnn.pack_padded_sequence(sent_emb, sent_len, batch_first=True)
        packed_outputs, hidden = self.rnn(packed_embedded)
        rnn_outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        dropout = self.dropout(rnn_outputs[:,-1,:])
        outputs = self.fc(dropout)
        # outputs = self.sigmoid(outputs)


        return outputs
        #output: batch_size, max_sent_len, n_hidden
        #hidden: n_layer, batch_size, n_hidden 
        


In [0]:
from torch.optim import Adam
n_hidden = 128
n_layers = 1
dropout = 0.5
embedding_dim = 64
out_features = 1
n_vocab =len(txt_field.vocab)


# build model

model = Text_RNN(n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features).to(device)


In [0]:
# print model achitecture and total number of parameters

def count_parameters(model):
    temp = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model architecture:\n\n', model)
    print(f'\nThe model has {temp:,} trainable parameters')

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

        
model.apply(init_weights)
count_parameters(model)

The model architecture:

 Text_RNN(
  (emb): Embedding(38344, 64)
  (rnn): RNN(64, 128, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

The model has 2,478,977 trainable parameters


In [0]:
n_epochs = 10
val_loss = None
save_name = 'rnn_model'
eval_every =  len(train_iter)//5
total_step = len(train_iter)*n_epochs
criterion = nn.BCELoss()
lr = 0.005
optimizer = Adam(model.parameters(), lr=lr)
#start training
loss_his, acc_his = TRAIN(model,train_iter, val_iter,  n_epochs, 
      eval_every, total_step, criterion, optimizer, val_loss, device, save_name)

Epoch [1/10], Step [250/12500], Train Loss: 0.6958, Valid Loss: 0.6925
Train Acc : 0.49387502670288086 ; Validation Acc 0.49939998984336853
Model saved to ==> rnn_model
Epoch [1/10], Step [500/12500], Train Loss: 0.6948, Valid Loss: 0.6926
Train Acc : 0.5021250247955322 ; Validation Acc 0.5001999735832214
Epoch [1/10], Step [750/12500], Train Loss: 0.7030, Valid Loss: 0.6928
Train Acc : 0.5062500238418579 ; Validation Acc 0.5008000135421753
Epoch [1/10], Step [1000/12500], Train Loss: 0.6945, Valid Loss: 0.6931
Train Acc : 0.499625027179718 ; Validation Acc 0.49879997968673706
Epoch [1/10], Step [1250/12500], Train Loss: 0.6931, Valid Loss: 0.6932
Train Acc : 0.5007500052452087 ; Validation Acc 0.4989999830722809
Epoch [2/10], Step [1500/12500], Train Loss: 0.7554, Valid Loss: 0.6928
Train Acc : 0.5003750324249268 ; Validation Acc 0.5023999810218811
Epoch [2/10], Step [1750/12500], Train Loss: 0.6908, Valid Loss: 0.6925
Train Acc : 0.49925002455711365 ; Validation Acc 0.500400006771087

In [0]:
# print(loss_his.text[0][0])
# print(model(loss_his.text[0],loss_his.text[1]))
train.examples[0].text[0]

'i'

In [0]:


import matplotlib.pyplot as plt
def plot_graph(loss_his):
  y1 = loss_his['train']
  y2 = loss_his['val']
  x = range(len(loss_his[0]))

  print(best_val_loss)
  plt.plot(x,y1,label = "train loss")
  plt.plot(x,y2,label = "val loss")


  plt.title("Loss Graph")
  plt.legend()  
  plt.show() 

##Different RNN Setting


#####1. GRU

In [0]:
import torch.nn as nn

class Text_GRU(nn.Module):
    def __init__(self, n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features):
        super(Text_GRU, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_vocab, embedding_dim)
        self.rnn = nn.GRU(
                input_size=embedding_dim,
                hidden_size=n_hidden,
                num_layers=n_layers,
                batch_first=True,
                dropout = dropout
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_hidden, out_features)
        self.sigmoid = nn.Sigmoid()
    def forward(self, sent, sent_len):
        # sent: batch_size, max_sent_len
        # sent_len: batch_size
        # print(sent.dtype)
        sent_emb = self.emb(sent)  #batch_size, max_sent_len, embedding_dim
        # method 1
        # rnn_outputs, hidden = self.rnn(sent_emb)
        # print("here", outputs)
        # method 2, pack the input sequence, more computationally efficient
        packed_embedded = nn.utils.rnn.pack_padded_sequence(sent_emb, sent_len, batch_first=True)
        packed_outputs, hidden = self.rnn(packed_embedded)
        rnn_outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        dropout = self.dropout(rnn_outputs[:,-1,:])
        outputs = self.fc(dropout)
        outputs = self.sigmoid(outputs)


        return outputs
        #output: batch_size, max_sent_len, n_hidden
        #hidden: n_layer, batch_size, n_hidden 
        


In [0]:
from torch.optim import Adam
n_hidden = 128
n_layers = 1
dropout = 0.5
embedding_dim = 64
out_features = 1
n_vocab =len(txt_field.vocab)


# build model

model_1 = Text_GRU(n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features).to(device)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
n_epochs = 10
val_loss = None
save_name = 'gru_model'
eval_every = len(train_iter)//5
total_step = len(train_iter)*n_epochs
criterion = nn.BCELoss()
# lr = 0.005
optimizer = Adam(model_1.parameters())
#start training
loss_1_his, acc_1_his = TRAIN(model_1,train_iter,  val_iter,  n_epochs, 
      eval_every, total_step, criterion, optimizer, val_loss, device, save_name)

KeyboardInterrupt: ignored

In [0]:
plot_graph(loss_1_his)

TypeError: ignored

####2. LTSM

In [0]:
import torch.nn as nn

class Text_LSTM(nn.Module):
    def __init__(self, n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features, bidirectional):
        super(Text_LSTM, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(n_vocab, embedding_dim)
        self.rnn = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=n_hidden,
                num_layers=n_layers,
                bidirectional = bidirectional,
                batch_first=True,
                dropout = dropout
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_hidden, out_features)
        self.sigmoid = nn.Sigmoid()
    def forward(self, sent, sent_len):
        # sent: batch_size, max_sent_len
        # sent_len: batch_size
        # print(sent.dtype)
        sent_emb = self.emb(sent)  #batch_size, max_sent_len, embedding_dim
        # method 1
        # rnn_outputs, hidden = self.rnn(sent_emb)
        # print("here", outputs)
        # method 2, pack the input sequence, more computationally efficient
        packed_embedded = nn.utils.rnn.pack_padded_sequence(sent_emb, sent_len, batch_first=True)
        packed_outputs, hidden = self.rnn(packed_embedded)
        rnn_outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        dropout = self.dropout(rnn_outputs[:,-1,:])
        outputs = self.fc(dropout)
        outputs = self.sigmoid(outputs)


        return outputs
        #output: batch_size, max_sent_len, n_hidden
        #hidden: n_layer, batch_size, n_hidden 
        


In [19]:
n_hidden = 128
n_layers = 1
dropout = 0.5
embedding_dim = 64
out_features = 1
n_vocab =len(txt_field.vocab)


# build model
model_2 = Text_LSTM(n_vocab, embedding_dim, n_hidden, n_layers, dropout, out_features, False).to(device)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [20]:
from torch.optim import Adam
n_epochs = 10
val_loss = None
save_name = 'lstm_model'
eval_every = len(train_iter)//5
total_step = len(train_iter)*n_epochs
criterion = nn.BCELoss()
# lr = 0.005
optimizer = Adam(model_2.parameters())
#start training
loss_2_his, acc_2_his = TRAIN(model_2,train_iter,  val_iter,  n_epochs, 
      eval_every, total_step, criterion, optimizer, val_loss, device, save_name)


KeyboardInterrupt

